# netCDF module
Version 1.5.1.2

# 介绍
netcdf-python 是Python关于netCDF的一个库。
netCDF4添加了许多之前版本没有的特性，并且是在HDF5的基础上实现的。这个模块可以读取和写入新的netCDF4和旧的netCDF3格式，并且可以创建HDF5程序可读的文件。这个模块模仿[Scientific.IO.NetCDF](http://dirac.cnrs-orleans.fr/ScientificPython/)，并且该模块用户们应该熟悉。  
许多netCDF4特性已经实现，例如多个无限维度，组和zlib数据压缩。所有新的数据类型（如64位无符号的整形）都已经实现。Compound (struct), variable length (vlen) 和 enumerated (enum) 数据类型都支持，但是目前还不支持opaque 数据类型。混合的compound, vlen 和 enum 数据类型（如compound中包含enums或者vlen中包含compound）也不支持。

# 下载
* 来自github最新的代码。
* 最新版本(源代码和二进制安装程序)。

# 依赖环境
* Python 2.7 or later (python 3 works too).
* numpy array module, version 1.10.0 or later.
* Cython, version 0.21 or later.
* setuptools, version 18.0 or later.
* cftime for the time and date handling utility functions (num2date, date2num and date2index).
* The HDF5 C library version 1.8.4-patch1 or higher (1.8.x recommended) from . netCDF version 4.4.1 or higher is recommended if using HDF5 1.10.x - otherwise resulting files may be unreadable by clients using earlier versions of HDF5. For netCDF < 4.4.1, HDF5 version 1.8.x is recommended. Be sure to build with --enable-hl --enable-shared.
* Libcurl, if you want OPeNDAP support.
* HDF4, if you want to be able to read HDF4 "Scientific Dataset" (SD) files.
* The netCDF-4 C library from the github releases page. Version 4.1.1 or higher is required (4.2 or higher recommended). Be sure to build with --enable-netcdf-4 --enable-shared, and set CPPFLAGS="-I \\&#36; HDF5_DIR/include" and LDFLAGS="-L \\&#36;HDF5_DIR/lib", where\\&#36;HDF5_DIR is the directory where HDF5 was installed. If you want OPeNDAP support, add --enable-dap. If you want HDF4 SD support, add --enable-hdf4 and add the location of the HDF4 headers and library to \\&#36;CPPFLAGS and \\&#36;LDFLAGS.  
* for MPI parallel IO support, an MPI-enabled versions of the netcdf library is required, as is the mpi4py python module. Parallel IO further depends on the existence of MPI-enabled HDF5 or the PnetCDF library.  

# 安装
推荐使用anaconda安装：  
`conda install netCDF4`  
* install the requisite python modules and C libraries (see above). It's easiest if all the C libs are built as shared libraries.  
* By default, the utility nc-config, installed with netcdf 4.1.2 or higher, will be run used to determine where all the dependencies live.  
* If nc-config is not in your default \\&#36;PATH edit the setup.cfg file in a text editor and follow the instructions in the comments. In addition to specifying the path to nc-config, you can manually set the paths to all the libraries and their include files (in case nc-config does not do the right thing).
* run python setup.py build, then python setup.py install (as root if necessary).
* pip install can also be used, with library paths set with environment variables. To make this work, the USE_SETUPCFG environment variable must be used to tell setup.py not to use setup.cfg. For example, USE_SETUPCFG=0 HDF5_INCDIR=/usr/include/hdf5/serial HDF5_LIBDIR=/usr/lib/x86_64-linux-gnu/hdf5/serial pip install has been shown to work on an Ubuntu/Debian linux system. Similarly, environment variables (all capitalized) can be used to set the include and library paths for hdf5, netCDF4, hdf4, szip, jpeg, curl and zlib. If the libraries are installed in standard places (e.g. /usr or /usr/local), the environment variables do not need to be set.
* run the tests in the 'test' directory by running python run_all.py.

# 教程

## 创建、打开和关闭一个netCDF文件
使用python创建一个netCDF文件，你只需要使用`Dataset`构造器。这也是打开一个已存在的netCDF文件的方法。如果以读写模式（mode='w', 'r+' or 'a'）打开文件，你可以写入任何类型的数据，包括新的dimensions, groups, variables and attributes。netCDF文件有5中类型（NETCDF3_CLASSIC, NETCDF3_64BIT_OFFSET, NETCDF3_64BIT_DATA, NETCDF4_CLASSIC, and NETCDF4）。NETCDF3_CLASSIC是原始的netcdf二进制格式，限制2Gb以内的文件。默认的打开格式是NETCDF4。你可以检查data_model 属性确认打开的文件格式。当文件写入完成时，通过 `Dataset`中的 `close`关闭文件。  
例子如下：

In [1]:
from netCDF4 import Dataset
rootgrp = Dataset('data/test.nc','w',format='NETCDF4')
print(rootgrp.data_model)
rootgrp.close()

NETCDF4


远程OPeNDAP-hosted数据集可以通过HTTP获取，将`Dataset`中的文件名改成URL地址即可。这需要netCDF库支持OPenDAP，在构建时使用`--enable-drap`参数。
## netCDF文件中的组Groups
netCDF4支持层级结构的group组织数据，类似于文件系统中的目录结构。group充当variables, dimensions 和 attributes,及其它组的容器。`Dataset`创建一种特殊的组叫做'root group'，类似于unix系统中的根目录。要创建group实例，请使用`Dataset`或者`Group`实例的`createGroup`方法。`createGroup`包含一个参数，一个表示组名的python字符串。可以使用`Dataset`实例的`groups dictionary`属性按名称访问根组中包含的新组实例。只有NETCDF4格式支持组操作，如果你尝试使用NETCDF3创建一个组会得到错误信息。

In [2]:
rootgrp=Dataset('data/test.nc','a')
fctgrp=rootgrp.createGroup('forecasts')
analgrp=rootgrp.createGroup('analyses')
print(rootgrp.groups)

OrderedDict([('forecasts', <class 'netCDF4._netCDF4.Group'>
group /forecasts:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
), ('analyses', <class 'netCDF4._netCDF4.Group'>
group /analyses:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 
)])


Group可以存在于Dataset中的group里，就像unix文件系统中目录下还存在目录。每个group实例都有一个`groups`属性字典包含了该组内所有的group实例。每个group实例都有`path`属性包含类似于unix路径的组路径。为了简化嵌套组的创建，你可以使用类似unix的路径作为参数创建组。

In [3]:
fcstgrp1 = rootgrp.createGroup("/forecasts/model1")
fcstgrp2 = rootgrp.createGroup("/forecasts/model2")

如果中间的任何路径不存在它将会自动创建，就好比unix的命令`mkdir -p`。如果你试图创建任何已存在的组，不会提示错误，将会返回该存在的组。  
这是一个遍历所有`Dataset`下的group。python中的`walktree`迭代器又来遍历树节点。注意，print `Dataset`或者`Group`对象将会返回其内容的摘要信息。

In [4]:
def walktree(top):
     values = top.groups.values()
     yield values
     for value in top.groups.values():
         for children in walktree(value):
             yield children
print(rootgrp)
for children in walktree(rootgrp):
      for child in children:
          print(child)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: forecasts, analyses

<class 'netCDF4._netCDF4.Group'>
group /forecasts:
    dimensions(sizes): 
    variables(dimensions): 
    groups: model1, model2

<class 'netCDF4._netCDF4.Group'>
group /analyses:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 

<class 'netCDF4._netCDF4.Group'>
group /forecasts/model1:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 

<class 'netCDF4._netCDF4.Group'>
group /forecasts/model2:
    dimensions(sizes): 
    variables(dimensions): 
    groups: 



## netCDF文件中的维度Dimensions
netCDF使用维度定义所有变量的大小。因此在创建任何变量之前，需要先创建维度。一个特殊的情况，一般实际中不会出现，如一个没有维度的标量。使用`Dataset`和`Group`实例的`createDimension`方法创建维度。用python字符串来表示维度的名字，用整型来表示维度的大小。使用`None`或者0来创建无限维度（可以扩展的维度）。在这个例子中`time`和`level`都是无限的。拥有不止一个无限维度是netCDF4的特性，在netCDF3中， 只能有一个无限维度，而且必须作为第一个变量的维度。

In [5]:
level = rootgrp.createDimension('level',None)
time = rootgrp.createDimension('time',None)
lat = rootgrp.createDimension('lat',73)
lon = rootgrp.createDimension('lon',144)

所有的维度将储存在一个python字典。

In [6]:
print(rootgrp.dimensions)

OrderedDict([('level', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'level', size = 0
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 0
), ('lat', <class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 73
), ('lon', <class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 144
)])


调用python的`len`函数可以获取`Dimension`实例的当前长度，`Dimension`实例的`isunlimited`方法可以确认维度是否是无限或者可扩展的。

In [7]:
print(len(lon))
print(lon.isunlimited())
print(time.isunlimited())

144
False
True


直接输出`Dimension`对象可以获取概况信息。包含该维度的名字和长度以及是否是无限的。

In [8]:
for dimobj in rootgrp.dimensions.values():
    print(dimobj)

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'level', size = 0

<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 0

<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 73

<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 144



`Dimension`名字可以更改，使用`Dataset`和`Group`实例的`netCDF4.Dataset.renameDimension`方法。

## netCDF文件中的变量
netCDF变量就好比python中numpy的多维度矩阵。然而，和numpy矩阵不同的是，netCDF可以沿着不止一个无限维度扩展。使用`Dataset`和`Group`实例的`createVariable`方法创建netCDF变量。`createVariable`方法有两个必填参数，一是变量名字，二是变量的数据类型。变量的维度通过一个包含维度名字的tuple给出。创建一个标量，不填维度参数就可以了。变量基本数据类型对应于numpy数组的数据类型。  
维度也可以定义成变量，叫作坐标变量。`createVariable`方法会返回一个变量实例。

In [9]:
times = rootgrp.createVariable('time','f8',('time',))
levels = rootgrp.createVariable('level','i4',('level',))
latitudes = rootgrp.createVariable('lat','f4',('lat',))
longitudes = rootgrp.createVariable('lon','f4',('lon',))
temp = rootgrp.createVariable('temp','f4',('time','level','lat','lon',))

为了查看变量实例的概况，直接输出即可

In [10]:
print(temp)

<class 'netCDF4._netCDF4.Variable'>
float32 temp(time, level, lat, lon)
unlimited dimensions: time, level
current shape = (0, 0, 73, 144)
filling on, default _FillValue of 9.969209968386869e+36 used



你可以使用路径来一个组里创建变量

In [11]:
ftemp = rootgrp.createVariable('/forecasts/model1/temp','f4',('time','level','lat','lon',))

如果中间组不存在，那么将会自动创建。  
你也可以通过`Dataset`和`Group`路径查询`Dataset`和`Group`实例。

In [12]:
print(rootgrp['/forecasts/model1'])
print(rootgrp['/forecasts/model1/temp'])

<class 'netCDF4._netCDF4.Group'>
group /forecasts/model1:
    dimensions(sizes): 
    variables(dimensions): float32 temp(time,level,lat,lon)
    groups: 

<class 'netCDF4._netCDF4.Variable'>
float32 temp(time, level, lat, lon)
path = /forecasts/model1
unlimited dimensions: time, level
current shape = (0, 0, 73, 144)
filling on, default _FillValue of 9.969209968386869e+36 used



所有`Dataset`和`Group`里的变量都存储在python字典中。就和维度一样。

In [13]:
print(rootgrp.variables)

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
unlimited dimensions: time
current shape = (0,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('level', <class 'netCDF4._netCDF4.Variable'>
int32 level(level)
unlimited dimensions: level
current shape = (0,)
filling on, default _FillValue of -2147483647 used
), ('lat', <class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
unlimited dimensions: 
current shape = (73,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('lon', <class 'netCDF4._netCDF4.Variable'>
float32 lon(lon)
unlimited dimensions: 
current shape = (144,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('temp', <class 'netCDF4._netCDF4.Variable'>
float32 temp(time, level, lat, lon)
unlimited dimensions: time, level
current shape = (0, 0, 73, 144)
filling on, default _FillValue of 9.969209968386869e+36 used
)])


变量可以通过`Dataset`中的`renameVariable`方法重新命名。  
## netCDF文件的属性
netCDF文件中有两种类型的属性，全局和变量。全局属性提供组或者整个数据的信息。变量属性提供组里变量的属性。通过为`Dataset`和`Group`实例变量分配值来设置全局属性。变量的属性可以通过`Variable`实例的值来设定。属性可以是字符串、数字或者序列。以下是一个例子：

In [14]:
import time
rootgrp.description = 'bogus example script'
rootgrp.history = 'Created'+time.ctime(time.time())
rootgrp.source = 'netCDF4 python module tutorial'
latitudes.units = 'degrees north'
longitudes.units = 'degrees east'
levels.units = 'hPa'
temp.units = 'K'
times.units = 'hour since 0001-01-01 00:00:00.0'
times.calendar = 'gregorian'

`Dataset`、`Group`或者`Variable`实例中的`__dict__`属性提供netCDF4文件中所有属性的名字和值。

In [15]:
print(rootgrp.__dict__)

OrderedDict([('description', 'bogus example script'), ('history', 'CreatedWed May  8 15:35:37 2019'), ('source', 'netCDF4 python module tutorial')])


可以使用python`del`语句从`Dataset`、`Group`中删除某个属性。如`del grp.foo`

In [16]:
del rootgrp.history

## 从netCDF文件中写入或检索数据
现在你已经有一个netCDF变量实例，怎么讲数据写入呢？你只需要像操作数组那样操作它。

In [17]:
import numpy as np
lats = np.arange(-90,91,2.5)
lons = np.arange(-180,180,2.5)
latitudes[:] = lats
longitudes[:] = lons
print("latitudes = \n",latitudes[:])

latitudes = 
 [-90.  -87.5 -85.  -82.5 -80.  -77.5 -75.  -72.5 -70.  -67.5 -65.  -62.5
 -60.  -57.5 -55.  -52.5 -50.  -47.5 -45.  -42.5 -40.  -37.5 -35.  -32.5
 -30.  -27.5 -25.  -22.5 -20.  -17.5 -15.  -12.5 -10.   -7.5  -5.   -2.5
   0.    2.5   5.    7.5  10.   12.5  15.   17.5  20.   22.5  25.   27.5
  30.   32.5  35.   37.5  40.   42.5  45.   47.5  50.   52.5  55.   57.5
  60.   62.5  65.   67.5  70.   72.5  75.   77.5  80.   82.5  85.   87.5
  90. ]


和numpy不同，如果您将数据分配到当前定义的索引范围之外，则具有无限维度的netCDF变量对象将沿着这些维度增长。

In [18]:
nlats = len(rootgrp.dimensions['lat'])
nlons = len(rootgrp.dimensions['lon'])
print('temp shape before adding data = ',temp.shape)
temp[0:5,0:10,:,:] = np.random.uniform(size=(5,10,nlats,nlons))
print('temp shape after adding data = ',temp.shape)
print('levels shape after adding pressre data = ',levels.shape)

temp shape before adding data =  (0, 0, 73, 144)
temp shape after adding data =  (5, 10, 73, 144)
levels shape after adding pressre data =  (10,)


请注意，当沿着temp变量的level维度添加数据时，levels变量的维度也会跟着增加，虽然还没有往里填入数据。

In [19]:
levels[:] = [1000.,850.,700.,500.,300.,250.,200.,150.,100.,50.]

虽然，Numpy和netCDF的切片规则有一些不同，但是基本一致。需要注意的是虽然netCDF支持布尔索引，但是只能支持一个维度的布尔值。

In [20]:
temp[0, 0, [0,1,2,3], [0,1,2,3]]

masked_array(
  data=[[0.9379823 , 0.53990453, 0.62666005, 0.9069424 ],
        [0.9756705 , 0.16037382, 0.20512678, 0.61722106],
        [0.726117  , 0.54609185, 0.3154905 , 0.92369676],
        [0.85004926, 0.05542484, 0.44301683, 0.5743179 ]],
  mask=False,
  fill_value=1e+20,
  dtype=float32)

In [21]:
a=temp[0, 0, [0,1,2,3], [0,1,2,3]]
print(a[np.array([[True,False,True,False],[True,False,True,False],[True,False,True,False],[True,False,True,False]])])
print(a[0,:])

[0.9379823  0.62666005 0.9756705  0.20512678 0.726117   0.3154905
 0.85004926 0.44301683]
[0.9379823  0.53990453 0.62666005 0.9069424 ]


In [22]:
tempdat = temp[::2, [1,3,6], lats>0, lons>0]
tempdat.shape

(3, 3, 36, 71)

## 处理时间坐标
时间坐标给netCDF用户带来特殊的挑战。大多数元数据标准（如CF）指定，时间应该使用特定相对于固定日期的日历，单位指定为YY-MM-DD hh:mm:ss之后的小时数。如果没有将值转换为日历日期或者从日历日期转换过来的程序，这种单位将很难处理。名叫`num2date`和`date2num`的函数提供这个功能。以下是一个使用它们的例子。

In [23]:
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num
dates = [datetime(2001,3,1)+n*timedelta(hours=12) for n in range(temp.shape[0])]
times[:] = date2num(dates, units=times.units, calendar=times.calendar)
print('time values (in units %s):' %times.units+'\n',times[:])
dates = num2date(times[:],units=times.units,calendar=times.calendar)
print('dates corresponding to time values:\n',dates)

time values (in units hour since 0001-01-01 00:00:00.0):
 [17533104. 17533116. 17533128. 17533140. 17533152.]
dates corresponding to time values:
 [real_datetime(2001, 3, 1, 0, 0) real_datetime(2001, 3, 1, 12, 0)
 real_datetime(2001, 3, 2, 0, 0) real_datetime(2001, 3, 2, 12, 0)
 real_datetime(2001, 3, 3, 0, 0)]


`num2date`将指定单位和日历中的时间数值转换为`datetime`对象，而`date2time`执行相反的操作。支持单签在CF元数据中定义的所有日历。还提供了一个名叫`date2index`的函数，该函数返回一系列datetime实例对应的时间变量索引。  
## 从多文件中读取netCDF数据
如果你需要从多个文件中读取netCDF数据，你可以使用`MFDataset`类读取数据，即使只含有一个文件。不要使用单个文件名来创建数据集实例，而是使用文件名列表或通配符字符串(然后使用python glob模块将其转换为已排序的文件列表)创建MFDataset实例。共享相同无限维度的文件列表中的变量聚合在一起，可以跨多个文件进行切片。为了阐述这点，让我们先创建含有相同变量的一串netCDF文件（有相同的无限维度）。目前还不支持NETCDF4格式的文件。

In [24]:
for nf in range(10):
    f = Dataset("data/mftest%s.nc" % nf,"w",format='NETCDF4_CLASSIC')
    f.createDimension("x",None)
    x = f.createVariable("x","i",("x",))
    x[0:10] = np.arange(nf*10,10*(nf+1))
    f.close()

现在来一次性读取它们。

In [25]:
from netCDF4 import MFDataset
f = MFDataset('data/mftest*.nc')
print(f.variables["x"][:])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


## 高效的压缩netCDF变量

In [28]:
temp1 = rootgrp.createVariable("data/temp1","f4",("time","level","lat","lon",))
temp2 = rootgrp.createVariable("data/temp2","f4",("time","level","lat","lon",),zlib=True)
temp3 = rootgrp.createVariable("data/temp3","f4",("time","level","lat","lon",),zlib=True,least_significant_digit=3)

后面的太高级了，懒得翻译了。